In [2]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import warnings
warnings.filterwarnings('ignore')

In [3]:
food_description = []
food_title = []
image_url = []
likes = []

restaurant_name = "mentai-ya-japanese-cuisine-sengkang"
driver = webdriver.Chrome('../util/chromedriver.exe')
restaurant_url = f"https://www.burpple.com/{restaurant_name}"
driver.get(restaurant_url)

prev = 0

while True:
    time.sleep(3)

    # Locate and counts number of food cards
    food_cards = driver.find_elements_by_class_name("feed-item")
    food_card_count = len(food_cards)

    # Locate and click load reviews button
    load_reviews_button = driver.find_element_by_id("load-more-reviews")
    driver.execute_script("arguments[0].click();", load_reviews_button)

    # Check if no new food cards
    if food_card_count == prev:
        # Get all food cards and images
        card_bodies = driver.find_elements_by_class_name("card-body")
        food_images = driver.find_elements_by_class_name('food-image')

        # Check if same length
        assert len(card_bodies) == len(food_images), "Rendering error"

        # Extract information
        for index, card_body in enumerate(card_bodies):
            food_title.append(card_body.find_elements_by_class_name("food-description-title")[0].text)
            try:
                food_description.append(card_body.find_elements_by_class_name("food-description-body")[0].text)
            except:
                food_description.append("")

            image_url.append(food_images[index].find_element_by_tag_name("img").get_attribute("data-srcset").split(" ")[0])
            try:
                likes.append(int(card_body.find_elements_by_class_name("food-activity-like")[0].find_element_by_tag_name("span").text.split(" ")[0]))
            except:
                likes.append(0)

        driver.quit()
        break
    else:
        prev = food_card_count
        
burpple_reviews_df = pd.DataFrame(
{'title': food_title,
 'description': food_description,
 'image_url': image_url,
 'likes' : likes
})

output_path = "../data/competitors/mentaiya/mentai_sengkang_burrple_reviews.csv"
burpple_reviews_df.to_csv(output_path)
print(f"Reviews output to {output_path}")